## 课程目标

- 进一步深入理解 NN 在 NLP 中的应用，及 Tensorflow 搭配的一些工具
- 了解 CNN 在图像和 NLP 处理上的应用
- CNN 相对普通 NN 的优势

## Softmax 性质再探究


- Softmax：分类问题标配激活函数
- 与 Sigmoid 区别（二分类问题）
  - 本质没有任何区别，可以推导过去
  - Sigmoid 输出节点是 1 个，Softmax 是 2 个
  - Softmax 几个输出
  

【卡片提示】可以自己画图感受一下

## 情感分类问题 revist

- NN 建模分类问题的套路：系数特征 + 词向量 + 隐层 + Softmax

- Word embedding + 隐层（不激活）的问题：其实没有意义
  - Word embedding + 隐层（不激活）和只用一个 embedding 没有区别


- 疑问
  - 12:55 用逻辑回归来处理情感分类问题就是把词向量的维度降到 1？

【embedding 这块感觉有点不好理解】


#### 作业1

- 用神经网络建模，处理之前的情感分类问题

## 模型 Debug 及 Tensorboard

In [ ]:
# 多久会重新读取一次目录中的日志，如果不设置的话即使刷新也不会立即出现
# logdir 表示日志写到哪里
# 该命令会直接启动 tensorboard
tensorboard --reload_interval=1 --logdir=./

In [1]:
import tensorflow as tf

In [44]:
tf.reset_default_graph()

In [45]:
c = tf.constant([1,2,3])
c2 = c + c

In [46]:
# 断言 c2 的每个元素都大于0，如果不是程序就终止
# 保障程序正确性的手段
assert_op = tf.Assert(tf.reduce_all(c2>0), [c2], name="assert")

In [47]:
# control_dependencies 类似于 if
# 用 out 代替 c2，就保证断言会被执行
# 断言看成 out 的一种输入，输入是 c2 的返回值
# 虚线代表控制流
# identity 返回的就是 c2，就是一个 operation，否则 out 和 c2 连不起来
with tf.control_dependencies([assert_op]):
    out = tf.identity(c2, name='out')

In [73]:
# 只声明 Tensor，没有 Operation 就不会画出来
tf.summary.FileWriter("./tf_log", graph=tf.get_default_graph())

### 关于 Tensorflow

- Nodes 就是 operations
- Edges 就是 tensors，表示 data 从一个节点到另一个节点

### 简化 tensorboard

In [68]:
tf.reset_default_graph()

In [69]:
# name_scope 会把所有的操作包起来
# 建议对变量命名

with tf.name_scope('hidden_layer'):
    
    m = tf.random_uniform((3, 3), -1, 1, name='weights')
    b = tf.zeros((1, 3), name='bias')
    input_ = tf.constant([[1, 2, 3]], dtype=tf.float32)

    raw_output = tf.matmul(input_, m) + b

In [70]:
with tf.name_scope('hidden_layer'):
    
    m2 = tf.random_uniform((3, 3), -1, 1, name='weights')
    b2 = tf.zeros((1, 3), name='bias')
    input_ = tf.constant([[1, 2, 3]], dtype=tf.float32)

    raw_output2 = tf.matmul(raw_output, m2) + b2

### 可视化

In [74]:
%matplotlib inline
from matplotlib import pyplot as plt

In [76]:
import numpy as np

In [120]:
tf.reset_default_graph()

In [121]:
x = np.arange(0, 20)
y = x + np.random.normal(size=20)

In [122]:
w = tf.Variable(tf.random_uniform([1], -1.0, 1.0), name='W')
b = tf.Variable(tf.zeros([1]), name='b')

y_bar = w * x + b
cost = tf.reduce_mean((y_bar - y) ** 2)

# 记录器记录 cost
# scalar 标量，只能是一个值
# 实际 run 时才会记录
cost_summary = tf.summary.scalar('cost', cost)

# 直方图
w_summary = tf.summary.histogram('w', w)


# mearge 会自动收集 graph 上所有 summary 操作
merged = tf.summary.merge_all()

In [125]:
# 不显示 graph
writer = tf.summary.FileWriter("./tf_log")#, graph=tf.get_default_graph())

In [124]:
train_step = tf.train.GradientDescentOptimizer(0.0001).minimize(cost)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        #_, cost_summary_run, w_run = sess.run([train_step, cost_summary, w_summary])
        
        # 用 merge
        _, merge_summary= sess.run([train_step, merged])
        writer.add_summary(merge_summary, i)
        
        #writer.add_summary(cost_summary_run, i)
        #writer.add_summary(w_run, i)
        #print(cost_summary_run)

In [95]:
tf.summary.FileWriter("./tf_log", graph=tf.get_default_graph())

- "tf.train.SummaryWriter": "tf.summary.FileWriter",
- "tf.scalar_summary": "tf.summary.scalar",
- "tf.histogram_summary": "tf.summary.histogram",
- "tf.audio_summary": "tf.summary.audio",
- "tf.image_summary": "tf.summary.image",
- "tf.merge_summary": "tf.summary.merge",
- "tf.merge_all_summaries": "tf.summary.merge_all",

#### 作业2

- 将 NN Language Model 的 cost 下降情况用 summary + tensorboard 的方式进行可视化
- 收集 Word embedding 的 histogram 并进行可视化

## 卷积

### 一维卷积

- 对信号的一种处理
- 元素个数：length(list) - length(window_size) + 1
  - window 就是 kernel，x y z
  - list 就是输入
- 填充：希望结果的个数和原始输入的个数一样，保证维度没有发生变化

### 二维卷积

- 图像模糊就相当于一维数据的平滑
- 模糊就是之前黑白分明的图像被平滑了

## 卷积神经网络

- 卷积的 Kernel 是 【学习】出来的
  - 只需要指定网络结构
  - 自动学习（自适应）最佳的中间表示
- 很多 Kernel 和 Pooling 的连续组合
- 卷积在图像上的好处
  - 参数个数少
  - 平移不变性（translation invariant）
- 卷积在自然语言处理上的好处
  - 不再局限于一个固定的【窗口】而是能综合考虑整个句子
  - 平移不变性

- input：输入，四维
- filter：Kernel
- strides：步长
- padding：最边缘的地方做填充，否则会比原图小一些

#### 作业3

- 根据作业框架，使用一个能够平滑图像的二维卷积对图像进行处理

【论文】

#### 作业4

- 应用卷积神经网络实现情感分类
  - 标准化格式，能够读取格式固定的训练、测试数据，跑出 Accuracy 和 Confusion Matrix 结果